In [ ]:
# pip  install tesseract-ocr
# pip install pytesseract
# pip install opencv-python-headless

In [ ]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 22.2 MB/s eta 0:00:00


In [ ]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import easyocr
import pandas as pd
import re
from google.colab import drive
import os
from tqdm import tqdm
drive.mount('/content/gdrive')

def enhance_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(1.1)  # Moderate contrast enhancement

def crop_img(img, scale=0.15):
    height_scaled = img.shape[0] - img.shape[0] * scale
    top_y = height_scaled
    img_cropped = img[int(top_y):, :]
    return img_cropped

def preprocess_image(image):
    grayscaleImage = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    #laplacian = cv2.Laplacian(grayscaleImage, cv2.CV_64F)
    #sharpened = cv2.filter2D(grayscaleImage, -1, kernel)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(grayscaleImage, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    enhanced=enhance_contrast(Image.fromarray(img))
    return enhanced

def extract_bottom_text(image_path):

    inputImage = cv2.imread(image_path)
    cropped_image=crop_img(inputImage)
    preprocessed_image = preprocess_image(cropped_image)

    # Save the preprocessed image for debugging (optional)
    preprocessed_image.save('preprocessed_image.png')

    return preprocessed_image

def convert_to_series(filename,ocr_output,df):
    split=filename.replace(".jpg","").split("_")
    text=ocr_output[0][-1].lower()

def find_max_length_output(ocr_output):
    text=""
    try:
      for value in ocr_output:
        if len(text)<len(value[-1]):
          text=value[-1]
    except:
      text="Unreadable"
    return text

def convert_to_series(filename,ocr_output,df):
    split=filename.replace(".jpg","").split("_")
    ocr_output=find_max_length_output(ocr_output)
    text=ocr_output.lower()

    # Get Kelurahan
    try:
      kelurahan=re.findall(r"\b(?!jl\.)(\w+(?:\s\w+)*)(?=\s*[,;]\s*kec\.|\s*kec\.)", text)[0]
    except:
      try:
        kelurahan=re.findall(r"\b(?!jl\.)(\w+(?:\s\w+)*)(?=\s*[,;]\s*kec\:|\s*kec\:)", text)[0]
      except:
        kelurahan="Unreadable"

    # Get Kecamatan
    try:
      kecamatan=re.findall(r"kec\.\s(\w+)(?=[,;])", text)[0]
    except:
      try:
        kecamatan=re.findall(r"kec\:\s(\w+)(?=[,;])", text)[0]
      except:
        kecamatan="Unreadable"

    # Get Kabupaten Kota
    try:
      kabupaten_kota=re.search(r'(kabupaten|kota)\s(\w+)(?=[,;])', text).group(0)
    except:
      kabupaten_kota="Unreadable"

    # Get Postal Code
    try:
      postal_code=re.findall(r'jawa barat\s(\d{5})', text)[0]
    except:
      try:
        postal_code=re.findall(r'jawa barai\s(\d{5})', text)[0]
      except:
        postal_code="Unreadable"

    # Get Latitude
    try:
      lat=re.findall(r'lat:\s*(-?\d+\.\d+)', text)[0]
    except:
      lat="Unreadable"

    # Get Longitude
    try:
      long=re.findall(r'long:\s*(-?\d+\.\d+)', text)[0]
    except:
      long="Unreadable"

    dict = {"user_id":split[3],
    "filename": filename,
    #"alamat": re.findall(r"(.+?)(?=[,;])", var[0][-1])[0],
    "kelurahan": kelurahan,
    "kecamatan": kecamatan,
    "kabupaten_kota":  kabupaten_kota,
    "provinsi": "jawa barat",
    "postal_code": postal_code,
    "negara": "indonesia",
    "tanggal":split[-2] ,
    "waktu": split[-1],
    "lat": lat,
    "long": long,
    "output_ocr":'"'+text+'"'}
    df = pd.concat([df, pd.Series(dict).to_frame().T])
    return df

# Get the list of all files and directories from local
# path = "dataset/"
# directory_files = os.listdir(path)
reader= easyocr.Reader(['id'], gpu = True)

#filenames=pd.read_csv("readlist.txt", sep=" ", header=None)
df=pd.DataFrame(columns=["user_id","filename","kelurahan","kecamatan","kabupaten_kota","provinsi","postal_code","negara","tanggal","waktu","lat","long","output_ocr"])
directory_files = os.listdir("/content/gdrive/My Drive/BGR/5001-5050")
for filename in tqdm(directory_files,total=len(directory_files)):
  image_path="/content/gdrive/My Drive/BGR/5001-5050/"
  # image_path="dataset/" # -- for Local
  #print (image_path+filename)
  image_pil=extract_bottom_text(image_path+filename)
  bound=reader.readtext(np.asarray(image_pil),paragraph=True)
  df= convert_to_series(filename,bound,df)
df.to_excel("output_5001_5050.xlsx",index=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
pip install tesseract-ocr
pip install pytesseract
pip install opencv-python-headless

SyntaxError: invalid syntax (<ipython-input-12-f8dca02f9fe5>, line 1)

In [ ]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 13.4 MB/s eta 0:00:00


In [ ]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import easyocr
import pandas as pd
import re
from google.colab import drive
import os
from tqdm import tqdm
import time
drive.mount('/content/gdrive')

def enhance_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(1.1)  # Moderate contrast enhancement

def crop_img(img, scale=0.15):
    height_scaled = img.shape[0] - img.shape[0] * scale
    top_y = height_scaled
    img_cropped = img[int(top_y):, :]
    return img_cropped

def preprocess_image(image):
    grayscaleImage = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    #laplacian = cv2.Laplacian(grayscaleImage, cv2.CV_64F)
    #sharpened = cv2.filter2D(grayscaleImage, -1, kernel)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(grayscaleImage)



    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Optionally, invert the image if the text is lighter than the background

    img = cv2.bitwise_not(img)

    kernel = np.ones((1,1),np.uint8)
    #img = cv2.dilate(img, kernel, iterations=1)
    #img = cv2.erode(img, kernel, iterations=1)
    #kernel = np.ones((2, 2), np.uint8)
    #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    threshold, thresh = cv2.threshold(img, 15, 255, cv2.THRESH_BINARY)
    img = cv2.dilate(thresh, kernel, iterations=5)
    #img = cv2.erode(thresh, kernel, iterations=1)
    #kernel = np.ones((2, 2), np.uint8)
    #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    enhanced=enhance_contrast(Image.fromarray(img))
    return enhanced

def extract_bottom_text(image_path):

    inputImage = cv2.imread(image_path)
    cropped_image=crop_img(inputImage)
    preprocessed_image = preprocess_image(cropped_image)

    # Save the preprocessed image for debugging (optional)
    #preprocessed_image.save('preprocessed_image.png')

    return preprocessed_image

def find_max_length_output(ocr_output):
    text=""
    try:
      for value in ocr_output:
        if len(text)<len(value[-1]):
          text=value[-1]
    except:
      text="Unreadable"
    return text

def convert_to_series(filename,ocr_output,df):
    split=filename.replace(".jpg","").split("_")
    ocr_output=find_max_length_output(ocr_output)
    text=ocr_output.lower()

    # Get Kelurahan
    kel_patterns = [
        r'(\w+\s\w+)(?=\s(kec))',
        r'(\w+)(?=\s(kec))',
        r'(\w+\s\w+)(?=(:kec|,kec|.kec|;kec|: kec|, kec|. kec|; kec| : kec| , kec| . kec| ; kec))',
        r'(\w+)(?=(:kec|,kec|.kec|;kec|: kec|, kec|. kec|; kec| : kec| , kec| . kec| ; kec))'
    ]

    for pattern in kel_patterns:
        match = re.findall(pattern, text)
        if match:
            kelurahan = ' '.join(match[0][:-1])
            break
    else:
        kelurahan = 'Unreadable'

    # Get Kecamatan
    kec_patterns = [
        r'(kec.|kec|kec:|kec:.)\s(\w+)\s(\w+)(?=\s(,|;|:|kab|kabupaten|kota))',
        r'(kec.|kec|kec:|kec:.)\s(\w+)(?=\s(,|;|:|kab|kabupaten|kota))',
        r'(kec.|kec|kec:|kec:.)\s(\w+)\s(\w+)(?=(,|;|:|kab|kabupaten|kota))',
        r'(kec.|kec|kec:|kec:.)\s(\w+)(?=(,|;|:|kab|kabupaten|kota))',
        r'(kec_)(\w+)(?=(,|;|:|kab|kabupaten|kota))'
    ]

    for pattern in kec_patterns:
        match = re.findall(pattern, text)
        if match:
            kecamatan = ' '.join(match[0][1:-1])
            break
    else:
        kecamatan = 'Unreadable'

    # Get Kabupaten Kota
    kab_kota_patterns = [
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)\s(\w+)\s(\w+)(?=\s+(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)\s(\w+)(?=\s+(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)\s(\w+)\s(\w+)(?=(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)\s(\w+)(?=(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)(\w+)\s(\w+)(?=\s+(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)(\w+)(?=\s+(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)(\w+)\s(\w+)(?=(,|;|:|jawa))',
        r'(kabupaten|kabuppaten|kota|kab.|kab. |kab .|kab:|kab_)(\w+)(?=(,|;|:|jawa))'
    ]

    for pattern in kab_kota_patterns:
        match = re.findall(pattern, text)
        if match:
            kabupaten_kota = ' '.join(match[0][1:-1])
            break
    else:
        kabupaten_kota = 'Unreadable'

    # Get City Type
    if match:
        city_type = "kabupaten" if any('kab' in word for word in match[0]) else "kota"
    else:
        city_type = "Unreadable"

    # Get Postal Code
    try:
      postal_code=re.findall(r'\b\d{5}\b', text)[0]
    except:
      try:
        postal_code=re.findall(r'jawa barai\s(\d{5})', text)[0]
      except:
        postal_code="Unreadable"

    # Get Latitude
    try:
      lat=re.search(r'lat[.,;:|]?\s*(-?\d+\.\d+)', text, re.IGNORECASE).group(1)
    except:
      lat="Unreadable"

    # Get Longitude
    try:
      long=re.search(r'long[.,;:|]?\s*(-?\d+\.\d+)', text, re.IGNORECASE).group(1)
    except:
      long="Unreadable"


    dict = {"user_id":split[3],
    "filename": filename,
    #"alamat": re.findall(r"(.+?)(?=[,;])", var[0][-1])[0],
    "kelurahan": kelurahan,
    "kecamatan": kecamatan,
    "city_type": city_type,
    "kabupaten_kota":  kabupaten_kota,
    "provinsi": "jawa barat",
    "postal_code": postal_code,
    "negara": "indonesia",
    "tanggal":split[-2] ,
    "waktu": split[-1],
    "lat": lat,
    "long": long,
    "output_ocr":'"'+text+'"'}
    df = pd.concat([df, pd.Series(dict).to_frame().T])
    return df

reader= easyocr.Reader(['id'], gpu = True)
#filenames=pd.read_csv("readlist.txt", sep=" ", header=None)
df=pd.DataFrame(columns=["user_id","filename","kelurahan","kecamatan","city_type","kabupaten_kota","provinsi","postal_code","negara","tanggal","waktu","lat","long","output_ocr"])
directory_files = os.listdir("/content/gdrive/My Drive/BGR/40001-60000/")
jpg_files = [s for s in directory_files if s.endswith(".jpg")]
for filename in tqdm(jpg_files,total=len(jpg_files)):
  image_path="/content/gdrive/My Drive/BGR/40001-60000/"
  image_pil=extract_bottom_text(image_path+filename)
  bound=reader.readtext(np.asarray(image_pil),paragraph=True)
  df= convert_to_series(filename,bound,df)
df.to_excel("output_40001_60000.xlsx",index=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma